In [1]:
import pandas as pd
path = "/content/drive/MyDrive/ML project - Nurshat/fb_scraped.csv"
scraped_df = pd.read_csv(path)

In [2]:
!pip install -U -q "google"
!pip install -U -q "google.genai"

import os
from google.colab import userdata
from google.colab import drive
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")

drive.mount("/content/drive")
# Please ensure that uploaded files are available in the AI Studio folder or change the working folder.
os.chdir("/content/drive/MyDrive/Google AI Studio")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 8.2 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import os
from google import genai
from google.genai import types
import pandas as pd
import time

# Your existing generate function modified to accept text input
def generate(text):
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.5-flash"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=text),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=0.1,
        thinking_config = types.ThinkingConfig(
            thinking_budget=0,
        ),
        response_mime_type="text/plain",
        system_instruction=[
            types.Part.from_text(text="""## Task:
Determine whether the user provided social media post is related to **any real disaster/threat or not**. Options are: 1 (text: \"a disaster\") else 0 (text: \"not a disaster\")

## Examples:
I actually posted this by accident to twitter but have decided this can stay = 0
Why are Puerto Ricans refusing to go home? Aside from the 5.4 aftershock just now, the ground in southwest Puerto Rico has… = 1
I’m here on Hwy 278 near Thaxton where a home is completely engulfed in flames... Thaxton and Pontotoc emergency crews are… = 1
Each financial year money is allocated for disaster management, but we still raise paybills during famine and floods #UhuruAddress = 0
Telangana: Section 144 has been imposed in Bhainsa from January 13 to 15, after clash erupted between two groups on January 12. Po… = 1
Help for Animal Fire Victims Premier Annastacia Palaszczuk has announced grants for wildlife carers inundated with… https://t.co/x0sedOKDlg... = 0
#PUERTORICO: EVERTEC, yesterday announced that the Company will donate $50,000 for earthquake relief in Puerto Rico. = 0


## Output Format <only the corresponding number>:
1 or 0
"""),
        ],
    )

    try:
        for chunk in client.models.generate_content_stream(
            model=model,
            contents=contents,
            config=generate_content_config,
        ):
            return int(chunk.text.strip())  # Convert response to integer and return
    except Exception as e:
        print(f"Error processing text: {text[:50]}... Error: {str(e)}")
        return None  # Return None if there's an error



In [4]:
for text in scraped_df['Text']:
    test_sentence = text

    test_prediction = generate(test_sentence)

    print(f'Test sentence: "{test_sentence}" is {"a disaster" if test_prediction == 1 else "not a disaster"}')

    import spacy
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(test_sentence)
    locations = [ent.text for ent in doc.ents if ent.label_ == 'GPE']
    print("Disaster Locations:", locations)

Test sentence: "Did vou feel that earthquake? " is a disaster
Disaster Locations: []
Test sentence: "An estimated magnitude 4.2 earthquake was reported at 1:27 pm am on Tuesday afternoon, it’s epicenter 21 km from Kingston. " is a disaster
Disaster Locations: ['Kingston']
Test sentence: "Source: Sources: Android Earthquake Alerts System, ready.gov… See more" is a disaster
Disaster Locations: []
Test sentence: "Flooding in Catarman, Northern Samar 
 #Catarman #Samar #Flood Samar News" is a disaster
Disaster Locations: ['Catarman']
Test sentence: "#sponsored I'll confess that in the past, I've taken my gas-powered string trimmer into the garage to try to start it without my neighbors watching me struggle...Ha! Not anymore! Husqvarna has made my life much easier! Their battery operated Weed Eater (R) starts with the simple touch of a button. And it's lightweight and easy to use! It makes my outdoor projects even easier to maintain and enjoy. #HusqvarnaPartner #MaxBattery #outdoorliving #f